## Fundamentals of Data Science: Final Presentation

<center><img src="Zoom_596.png" alt="drawing" width="900"/></center>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from astropy.time import Time
from collections import Counter

sns.set_context("talk")
sns.set_style("white")
sns.set_palette("Dark2")

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import logging
logger = logging.getLogger("pymc3")
logger.propagate = False

*Charmi*
## Supernovae: Why Should We Care
<center><img src="SN1994D.jpg" alt="drawing" width="500"/></center>


Supernovae are coooool beans, and for that reason alone we'd want to study them. Picture here:

*Charmi*
# Objects in this project

* 2020oi - Type Ic
* LSQ12gdj - Type Ia
* 1987A - Type II
* 2011fe - Type Ia

Why did we choose these? We wanted a spread in data quality and types of supernovae. 1987A and 2011fe are well studied and good benchmarks. 2020oi is a newly discovered event that Alex is working on in real life*. LSQ12gdj was chosen because it is the main focus of Scalzo et. al. 2014, where the authors similarly obtain a bolometric light curve.

*real life is not class.

*Charmi*

## The Bolometric Problem
SN emit in nearly all wavelengths, but we aren't able to take continuous spectra. 


* Dust obscuration

* Missing some flux during rise

* Bands are discretized - UGRIZY

We need some way to estimate the amount of energy the SN puts out in all wavelengths while only measuring a few. 

*Cyrus*
## Gaussian Processes to the rescue! 
(With hierarchical models to boot)

*Cyrus*
## Part I: Creating a Mean Model
We opted for a spline of two quadratic functions. We attempted two different models: 
* Spline at maximum
* Spline during decine 

<center><img src="all_mean_models.png" alt="drawing" width="900"/></center>

*Cyrus*
## Part II: With George, we can condition on the data (with a squared exponential kernel) for a better fitting model. 

<center><img src="all_george_models.png" alt="drawing" width="900"/></center>

*Melanie*
## Part III: Gaussian Processes Implemented in PyMC3
So that we'll be able to use a hierarchical model!

One main difference between the GPs in george and PyMC3 is that george let us make the turning point a parameter. PyMC3 did not allow booleans from parameters, so we had to manually approximate this time value.

<center><img src="all_pymc3_models.png" alt="drawing" width="900"/></center>

*Melanie*
## Part IV: Hierarchical GPs in PyMC3
For $N$ bands, we generate six random variables for our quadratic parameters: 
* $N$-dimensional Gaussian distributions centered on polyfit values
* $N$ gps with same mean model, squared exponential kernel
* Condition on low-error values

The usefulness of a hierarchical model is that our parameters for each band could be drawn from the same distribution. Doing so shrinks the variance between the parameter estimates.

Other parameters for consideration were the smoothing length, noise, and the "turn over" time for the spline. After tweaking, we did end up drawing the kernel smoothing length from a distribution as well.

<center><img src="all_pymc3_allbands.png" alt="drawing" width="900"/></center>

*Alex*
## Converting Posteriors to Bolometric Lightcurves

* First we took the median of all posteriors in each band.
* The error was found by taking the standard deviation of the posteriors and adding them in quadrature across the bands.
* We used trapezoidal integration to get the bolometric flux, and from that, luminosity.

To convert from bolometric flux to bolometric luminosity, we use the following relationship: 

$$ L_{bol} = F_{bol}  4 \pi d_{SN}^2$$

Where $d_{SN}$ is the distance to the supernova.

<center><img src="all_luminosities.png" alt="drawing" width="900"/></center>

*Alex*
## Estimating the Temperature Evolution

If we assume a blackbody, then we can estimate the temperature of the supernova over time with the equation:

$$
L_{bol} = 4 \pi R^2 \sigma_{SB} T_{eff}^4
$$

Using this, we had to assume some radius based on the progenitor of our objects. It is important to note that the temperature here is the temperature of the photosphere (corresponding to the photosphere radius). 

<center><img src="all_temperatures.png" alt="drawing" width="900"/></center>

*Alex*
## Obtaining Temperature from a Blackbody Fit

Other people have made packages that estimate bolometric lightcurves (SuperBoL and superbol, for example). In superbol, they fit a blackbody curve to get temperature and radius. We did make an attempt to find temperature this way.

In [ ]:
<center><img src="all_bb_temp.png" alt="drawing" width="900"/></center>

*Charmi*
## Summary

Our goal was to get **temperatures** from supernova data using **Gaussian Processes** in a **hierarchical model**. To do so, we had to consult literature and our own astrophysical knowledge about what model parameters might make sense or what sources of error are important. Ta-da!